In [12]:
import os
%load_ext autoreload
%autoreload 2
import pandas as pd
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
os.environ['SPARK_HOME'] = 'C:/Users/saul2/Spark_DF/spark-3.5.5-bin-hadoop3'
os.environ['HADOOP_HOME'] = 'C:/Users/saul2/Spark_DF/spark-3.5.5-bin-hadoop3'
os.environ['JAVA_HOME'] = 'C:/Program Files/Java/jdk1.8.0_202'

In [16]:
spark = SparkSession.builder.appName("Practice").getOrCreate()

In [28]:
csv_file = 'Students_Grading_Dataset.csv'
df = spark.read.option("header", True).csv(csv_file)
df.createOrReplaceTempView("Students")

In [36]:
# Show Column Names
query = 'DESC STUDENTS'
result = spark.sql(query)
result.show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|          Student_ID|   string|   NULL|
|          First_Name|   string|   NULL|
|           Last_Name|   string|   NULL|
|               Email|   string|   NULL|
|              Gender|   string|   NULL|
|                 Age|   string|   NULL|
|          Department|   string|   NULL|
|      Attendance (%)|   string|   NULL|
|       Midterm_Score|   string|   NULL|
|         Final_Score|   string|   NULL|
|     Assignments_Avg|   string|   NULL|
|         Quizzes_Avg|   string|   NULL|
| Participation_Score|   string|   NULL|
|      Projects_Score|   string|   NULL|
|         Total_Score|   string|   NULL|
|               Grade|   string|   NULL|
|Study_Hours_per_Week|   string|   NULL|
|Extracurricular_A...|   string|   NULL|
|Internet_Access_a...|   string|   NULL|
|Parent_Education_...|   string|   NULL|
+--------------------+---------+-------+
only showing top

In [48]:
# Students Older than 21
query = "SELECT * FROM STUDENTS WHERE AGE > 21"
result = spark.sql(query)
#result.show()

In [84]:
# Students with a total_score higher than their department_avg
query = '''
    WITH DEPT_AVG AS (
        SELECT DEPARTMENT, ROUND(AVG(TOTAL_SCORE),2) AVG_SCORE
        FROM STUDENTS 
        GROUP BY DEPARTMENT
    )
    SELECT STUDENTS.DEPARTMENT, STUDENT_ID, FIRST_NAME, LAST_NAME, TOTAL_SCORE, DEPT_AVG.AVG_SCORE
    FROM STUDENTS, DEPT_AVG
    WHERE STUDENTS.DEPARTMENT = DEPT_AVG.DEPARTMENT
        AND STUDENTS.TOTAL_SCORE > DEPT_AVG.AVG_SCORE
    ORDER BY STUDENTS.DEPARTMENT, STUDENT_ID
    '''
result = spark.sql(query)
result.show()

+----------+----------+----------+---------+-----------+---------+
|DEPARTMENT|STUDENT_ID|FIRST_NAME|LAST_NAME|TOTAL_SCORE|AVG_SCORE|
+----------+----------+----------+---------+-----------+---------+
|  Business|     S1001|     Maria|    Brown|      92.29|     74.0|
|  Business|     S1006|     Ahmed|    Jones|      79.78|     74.0|
|  Business|     S1010|      John|    Smith|      74.52|     74.0|
|  Business|     S1013|      John| Williams|       74.8|     74.0|
|  Business|     S1016|     Maria|    Jones|      78.95|     74.0|
|  Business|     S1022|      Sara|    Davis|      87.98|     74.0|
|  Business|     S1026|      Emma|  Johnson|      89.76|     74.0|
|  Business|     S1030|     Ahmed|    Brown|      74.94|     74.0|
|  Business|     S1038|       Ali| Williams|      76.22|     74.0|
|  Business|     S1040|      John|  Johnson|       90.3|     74.0|
|  Business|     S1044|     Maria|    Smith|      88.66|     74.0|
|  Business|     S1053|     Maria|    Smith|      77.51|     7

In [86]:
result.write.mode("overwrite").saveAsTable("student_results_table")

Py4JJavaError: An error occurred while calling o137.saveAsTable.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.validateTableLocation(SessionCatalog.scala:419)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.run(createDataSourceTables.scala:176)
	at org.apac